In [1]:
%%capture                         
                                  # This capture statement is optional and you can delete it or put a hashtag in front of it.
                                  # Removing it will show all the "chatter" normally produced during import steps. 

# !pip install os
import os                         # os is a module for navigating your machine (e.g., file directories).
# !pip install wheel
# !pip install pandas 
import pandas as pd               # pandas is necessary for working with data frames - shortening it to pd just saves time. 
pd.set_option('display.max_colwidth', 200)   # some of the files are big so set a big column width. 
# !pip install numpy
import numpy as np                # like pandas, numpy is useful and useful to have a short name for
# !pip install statistics
import statistics                 # gotsta have stats
# !pip install matplotlib
import matplotlib as plt          # we are gonna want some plots


# !pip install csv
import csv                        # csv is for importing and working with csv files
import re

In [ ]:
# Sorry Jools, I rewrote this for my mac but just comment out my line when you run it!

print(os.listdir("../raw_data")  )     
# print(os.listdir("..\\raw_data")  )  # check 'raw_data' folder is not empty/has correct stuff and to know what is the correct file to import

In [ ]:
focus_group_respondents = pd.read_csv('../raw_data/focus_groups_0702.csv')  # import the file selected in the previous step
print(len(focus_group_respondents))                                         # check the length of the file
# focus_group_respondents.head(4)         

In [ ]:
print(focus_group_respondents.columns)                                                                     # a quick check to see what the column names look like
focus_group_respondents.columns = focus_group_respondents.columns.to_series().apply(lambda x: x.strip())   # renames the columns the same as what they were but with no trailing white space
print(focus_group_respondents.columns)    

In [ ]:
focus_group_respondents.head()

In [ ]:
responses = focus_group_respondents.loc[:, ['Finished', 'ResponseId',
                                            'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Africa', 'UK', 'Europe', 'Americas', 'Asia/Oceana', 'Q6.5', 'Q7','Q8']]

print(len(responses))

In [ ]:
responses.head()

In [ ]:
responses_cleaned = responses.rename(columns={"Q1": "Name", "Q2": "Organisation", "Q3": "Email", "Q4": "Country", "Q5": "TimeZone", "Q6.5": "Any_further_info", "Q7": "Pronouns", "Q8": "Accommodations"})

To sort:

* Drop any rows where key info is missing: name, email, org
* Drop those if they have an email without a name or org? Depends how many
There is no missing values for any of these columns

* Drop those with gmail accounts?
Without, we still have 45 respondents from Africa.

* Drop any rows that have no time slots booked and no comments in the "Other" column
Only 3 values but have been removed.

In [ ]:
gmail = responses_cleaned[responses_cleaned.Email.str.contains('gmail')].reset_index(drop = True)

In [ ]:
gmail.TimeZone.value_counts()

In [ ]:
# Let's remove these from our responses

responses_cleaned = responses_cleaned[~responses_cleaned.Email.str.contains('gmail')].reset_index(drop = True)

In [ ]:
# We still have a decent amount of responses from Africa

responses_cleaned.TimeZone.value_counts()

In [ ]:
# Let's get rid of those who answered "None of these dates work"

responses_cleaned = responses_cleaned[
    ~(
        (responses_cleaned['UK'].str.contains("None of these", na=False)) |
        (responses_cleaned['Europe'].str.contains("None of these", na=False)) |
        (responses_cleaned['Americas'].str.contains("None of these", na=False)) |
        (responses_cleaned['Asia/Oceana'].str.contains("None of these", na=False)) |
        (responses_cleaned['Africa'].str.contains("None of these", na=False))
    )
]

In [ ]:
VIPs = ['Irena Vipavc Brvar', 'Rajeev Samarage', 'Paula Lackie', 'Alita Nandi', 'Winny Nekesa AKULLO', 'San Cannon', 'Ricarda Braukmann', 'Deb Wiltshire', 'Olatunbosun Obileye', 'Wei Yin', 'Bobray Bordelon']

In [ ]:
# Let's create a column based on whether the respondent is on our VIP list

responses_cleaned['Invited'] = responses_cleaned['Name'].isin(VIPs).map({True: 'Yes', False: 'No'})

In [ ]:
# Let's sort values with our VIPs at the top

responses_cleaned = responses_cleaned.sort_values(by = 'Invited', ascending = False).reset_index(drop = True)

In [ ]:
# Combine all regional time slots into a single column per person
responses_cleaned["Available_Slots"] = responses_cleaned[['Africa', 'UK', 'Europe', 'Americas', 'Asia/Oceana']].apply(
    lambda row: [slot.strip() for slot in ', '.join(map(str, filter(None, row))).split('; ') if slot.strip()],
    axis=1
)

In [ ]:
all_time_slots = sorted(set(slot.strip() for col in ['Africa', 'UK', 'Europe', 'Americas', 'Asia/Oceana']
                            for slots in responses_cleaned[col].dropna().str.split(', ') for slot in slots))

In [ ]:
all_time_slots[:2]

In [ ]:
for slot in all_time_slots:
    responses_cleaned[slot] = "No"

In [ ]:
responses_cleaned